#1. Load the required libraries

#2. Read in the 5 datasets

In [10]:
setwd("/home/ayushi/Projects_2020/Naturalness/organized_InterSpeech_2022/scripts/")
#install.packages("itertools")
library(dplyr)
library(ggpubr)
library(tidyverse)
library(glue)
library(rstatix)
library(itertools)

CV_Merged = read.csv("../text_data/V.06.Merged.Vowels.CV.txt", sep = '\t', header = TRUE, na.strings = "NA")
VC_Merged = read.csv("../text_data/V.06.Merged.Vowels.VC.txt", sep = '\t', header = TRUE, na.strings = "NA")
#str(CV_Merged)

'data.frame':	15132 obs. of  64 variables:
 $ Filename              : Factor w/ 98 levels "booksent_2013_0001",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Word.Index            : int  0 0 0 0 0 0 0 0 0 0 ...
 $ Word                  : Factor w/ 488 levels "acceptance","accountable",..: 439 439 439 439 439 439 439 439 439 439 ...
 $ Phoneme.Index         : int  1 1 1 1 1 1 1 1 1 1 ...
 $ Vowel                 : Factor w/ 21 levels "AA1","AA2","AE0",..: 10 10 10 10 10 10 10 10 10 10 ...
 $ Obstruent             : Factor w/ 17 levels "B","CH","D","DH",..: 4 4 4 4 4 4 4 4 4 4 ...
 $ Vowel_Height          : Factor w/ 3 levels "High","Low","Medial": 2 2 2 2 2 2 2 2 2 2 ...
 $ Vowel_Frontness       : Factor w/ 3 levels "Back","Central",..: 3 3 3 3 3 3 3 3 3 3 ...
 $ Vowel_Type            : Factor w/ 1 level "Vowel": 1 1 1 1 1 1 1 1 1 1 ...
 $ Manner                : Factor w/ 3 levels "Affricate","Fricative",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ Voicing               : Factor w/ 2 levels "Voiced","Voiceless": 1 

In [11]:
feature_to_analyse = "Rel_Amp_F4"
group_type = "Sys.Fam.CrossType"
formula_1 = "CV_Merged[[feature_to_analyse]] ~ CV_Merged[[group_type]]"
formula_2 = CV_Merged[[group_type]]
#mod_formula = paste0('velocity_day_', to, ' ~ velocity_day_', from)
library(glue)
#CV_Merged = ungroup(CV_Merged)
dunn_test_results = CV_Merged %>% group_by(Manner) %>% dunn_test(formula=eval(parse(text=glue("{feature_to_analyse}~{group_type}"))), p.adjust.method = "BH", detailed = FALSE)
#dunn_test_results

dunn_test_indivsys = dunn_test_results[which(dunn_test_results$group1=='Natural' & (dunn_test_results$group2=='HMM-R2' | dunn_test_results$group2=='HMM-R3')),]
dunn_test_indivsys
nrow(CV_Merged[which(CV_Merged$Manner == "Affricate" & CV_Merged$Sys.Name == "M"),])
#pairwise_mean = Combined_VowelData_CV %>% group_by(Sys.Name) %>%  summarise_at(vars(Rel_Amp_F5), funs(median(., na.rm=TRUE), mean(., na.rm=TRUE)))
#pairwise_mean

Manner,.y.,group1,group2,n1,n2,statistic,p,p.adj,p.adj.signif
<fct>,<chr>,<chr>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<chr>


[1] 35

In [12]:
group_types = c("Sys.Name", "Sys.Fam", "Sys.Qual", "Sys.Fam.CrossType")
features_for_analyses = c("Vowel_Duration", "RMS_Amplitude", "Spectral.Tilt", "F0_onset", "F1_onset", "F2_onset", "F1_midpoint", "F2_midpoint", "F3_onset", "F4_onset", "F5_onset", "F3_midpoint", "F4_midpoint", "F5_midpoint", "Rel_Amp_F3", "Rel_Amp_F4", "Rel_Amp_F5", "F1_onset_dispersion", "F2_onset_dispersion", "F3_onset_dispersion", "F4_onset_dispersion", "F5_onset_dispersion", "F1_midpoint_dispersion", "F2_midpoint_dispersion", "F3_midpoint_dispersion", "F4_midpoint_dispersion", "F5_midpoint_dispersion", "Delta_F1", "Delta_F2", "Delta_F3", "Delta_F4", "Delta_F5")

#output_file_prefix = "../text_data/DTR_files/V.DTR" ### FOR NON-VOICING FACETS!!
output_file_prefix = "../DTR_files/voiced_vowels/V.DTR"

for (group in group_types) {
    Group_Results = data.frame(matrix(ncol = 12, nrow = 0))
    for (feature in features_for_analyses) {
        
        dunn_test_results = CV_Merged %>% group_by(Manner, Voicing) %>% dunn_test(formula=eval(parse(text=glue("{feature}~{group}"))), p.adjust.method = "BH", detailed = FALSE)
        dunn_test_results = as.data.frame(dunn_test_results)
        #View(dunn_test_results)
        #dunn_test_results$mean_diff = "NA"
        #dunn_test_results$median_diff = "NA"
        
        
        for (comp_item in (1:nrow(dunn_test_results))){
            
            reference = dunn_test_results$group1[comp_item]
            target = dunn_test_results$group2[comp_item]
            context = toString(dunn_test_results$Manner[comp_item])
            voice = toString(dunn_test_results$Voicing[comp_item])
            
            subset_manner_reference = CV_Merged[which(CV_Merged[["Manner"]] == context & CV_Merged[["Voicing"]] == voice & CV_Merged[[group]] == reference),]
            subset_manner_target = CV_Merged[which(CV_Merged[["Manner"]] == context & CV_Merged[["Voicing"]] == voice & CV_Merged[[group]] == target),]

            mean_difference = mean(subset_manner_target[[feature]], na.rm=TRUE) - mean(subset_manner_reference[[feature]], na.rm=TRUE)
            median_difference = median(subset_manner_target[[feature]], na.rm=TRUE) - median(subset_manner_reference[[feature]], na.rm=TRUE)
            dunn_test_results$mean_diff[comp_item] = round(mean_difference, 2)
            dunn_test_results$median_diff[comp_item] = round(median_difference, 2)
            #CV_Merged %>% group_by(Manner, reference)         
        }
        Group_Results = rbind(Group_Results, dunn_test_results)        
}

output = paste(output_file_prefix, group, "CV.txt", sep="_")
print(output)
message = paste("Writing to file about", group)
print(message)
write.table(Group_Results, file = output, row.names=FALSE, sep="\t")

}

[1] "Vowel_Duration"
[1] "RMS_Amplitude"
[1] "Spectral.Tilt"
[1] "F0_onset"
[1] "F1_onset"
[1] "F2_onset"
[1] "F1_midpoint"
[1] "F2_midpoint"
[1] "F3_onset"
[1] "F4_onset"
[1] "F5_onset"
[1] "F3_midpoint"
[1] "F4_midpoint"
[1] "F5_midpoint"
[1] "Rel_Amp_F3"
[1] "Rel_Amp_F4"
[1] "Rel_Amp_F5"
[1] "F1_onset_dispersion"
[1] "F2_onset_dispersion"
[1] "F3_onset_dispersion"
[1] "F4_onset_dispersion"
[1] "F5_onset_dispersion"
[1] "F1_midpoint_dispersion"
[1] "F2_midpoint_dispersion"
[1] "F3_midpoint_dispersion"
[1] "F4_midpoint_dispersion"
[1] "F5_midpoint_dispersion"
[1] "Delta_F1"
[1] "Delta_F2"
[1] "Delta_F3"
[1] "Delta_F4"
[1] "Delta_F5"
[1] "../DTR_files/voiced_vowels/V.DTR_Sys.Name_CV.txt"
[1] "Writing to file about Sys.Name"
[1] "Vowel_Duration"
[1] "RMS_Amplitude"
[1] "Spectral.Tilt"
[1] "F0_onset"
[1] "F1_onset"
[1] "F2_onset"
[1] "F1_midpoint"
[1] "F2_midpoint"
[1] "F3_onset"
[1] "F4_onset"
[1] "F5_onset"
[1] "F3_midpoint"
[1] "F4_midpoint"
[1] "F5_midpoint"
[1] "Rel_Amp_F3"
[1] "Rel

# Now we start the VC anlaysis:-

In [14]:
group_types = c("Sys.Name", "Sys.Fam", "Sys.Qual", "Sys.Fam.CrossType")
features_for_analyses = c("Vowel_Duration", "RMS_Amplitude", "Spectral.Tilt", "F0_offset", "F1_offset", "F2_offset", "F1_midpoint", "F2_midpoint", "F3_offset", "F4_offset", "F5_offset", "F3_midpoint", "F4_midpoint", "F5_midpoint", "Rel_Amp_F3", "Rel_Amp_F4", "Rel_Amp_F5", "F1_offset_dispersion", "F2_offset_dispersion", "F3_offset_dispersion", "F4_offset_dispersion", "F5_offset_dispersion", "F1_midpoint_dispersion", "F2_midpoint_dispersion", "F3_midpoint_dispersion", "F4_midpoint_dispersion", "F5_midpoint_dispersion", "Delta_F1", "Delta_F2", "Delta_F3", "Delta_F4", "Delta_F5")

#output_file_prefix = "../text_data/DTR_files/V.DTR"
output_file_prefix = "../DTR_files/voiced_vowels/V.DTR"

for (group in group_types) {
    Group_Results = data.frame(matrix(ncol = 12, nrow = 0))
    for (feature in features_for_analyses) {
        #print(feature)
        dunn_test_results = VC_Merged %>% group_by(Manner, Voicing) %>% dunn_test(formula=eval(parse(text=glue("{feature}~{group}"))), p.adjust.method = "BH", detailed = FALSE)
        dunn_test_results = as.data.frame(dunn_test_results)
        #View(dunn_test_results)
        dunn_test_results$mean_diff = "NA"
        dunn_test_results$median_diff = "NA"
        
        
        for (comp_item in (1:nrow(dunn_test_results))){
            
            reference = dunn_test_results$group1[comp_item]
            target = dunn_test_results$group2[comp_item]
            context = toString(dunn_test_results$Manner[comp_item])
            voice = toString(dunn_test_results$Voicing[comp_item])
       
            subset_manner_reference = VC_Merged[which(VC_Merged[["Manner"]] == context & VC_Merged[["Voicing"]] == voice & VC_Merged[[group]] == reference),]
            subset_manner_target = VC_Merged[which(VC_Merged[["Manner"]] == context & VC_Merged[["Voicing"]] == voice & VC_Merged[[group]] == target),]

            
            mean_difference = mean(subset_manner_target[[feature]], na.rm=TRUE) - mean(subset_manner_reference[[feature]], na.rm=TRUE)
            median_difference = median(subset_manner_target[[feature]], na.rm=TRUE) - median(subset_manner_reference[[feature]], na.rm=TRUE)
            dunn_test_results$mean_diff[comp_item] = round(mean_difference, 2)
            dunn_test_results$median_diff[comp_item] = round(median_difference, 2)
            #VC_Merged %>% group_by(Manner, reference)         
        }
        Group_Results = rbind(Group_Results, dunn_test_results)        
}

output = paste(output_file_prefix, group, "VC.txt", sep="_")
print(output)
message = paste("Writing to file about", group)
print(message)
write.table(Group_Results, file = output, row.names=FALSE, sep="\t")

}

[1] "../DTR_files/voiced_vowels/V.DTR_Sys.Name_VC.txt"
[1] "Writing to file about Sys.Name"
[1] "../DTR_files/voiced_vowels/V.DTR_Sys.Fam_VC.txt"
[1] "Writing to file about Sys.Fam"
[1] "../DTR_files/voiced_vowels/V.DTR_Sys.Qual_VC.txt"
[1] "Writing to file about Sys.Qual"
[1] "../DTR_files/voiced_vowels/V.DTR_Sys.Fam.CrossType_VC.txt"
[1] "Writing to file about Sys.Fam.CrossType"
